In [1]:
import weaviate
from weaviate.classes.init import Auth
import os
from dotenv import load_dotenv

weaviate_url = os.getenv("WEAVIATE_URL")
weaviate_key = os.getenv("WEAVIATE_API_KEY")
load_dotenv()

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url, auth_credentials=Auth.api_key(weaviate_key)
)

In [2]:
from weaviate.agents.personalization import PersonalizationAgent
from weaviate.classes.config import DataType
from helpers import PA_DEMO_COLLECTION

if PersonalizationAgent.exists(client, PA_DEMO_COLLECTION):
    pa = PersonalizationAgent.connect(client=client, reference_collection=PA_DEMO_COLLECTION)
else:
    pa = PersonalizationAgent.create(
        client=client,
        reference_collection=PA_DEMO_COLLECTION,
        user_properties={
            "age": DataType.NUMBER,
            "favorite_genres": DataType.TEXT_ARRAY,
            "favorite_years": DataType.NUMBER_ARRAY,
            "language": DataType.TEXT,
        },
    )

In [3]:
from weaviate.agents.classes import Persona
from weaviate.util import generate_uuid5
from uuid import uuid4  # If you want to generate a random UUID

persona_id = generate_uuid5("sebawita")  # To generate a deterministic UUID
# persona_id = uuid4()  # To generate a random UUID

if pa.has_persona(persona_id):
    print(f"Persona with ID {persona_id} already exists.")
else:
    print(f"Creating new persona with ID {persona_id}.")
    pa.add_persona(
        Persona(
            persona_id=persona_id,
            properties={
                "age": 30,
                "favorite_genres": ["Sci-Fi", "Horror", "Mystery"],
                "favorite_years": [1999, 1996, 2008, 2019],
                "language": "English",
            },
        )
    )

Persona with ID 3df23b31-0e9d-515b-9aef-ed704641bbe2 already exists.


In [5]:
pa.get_persona(persona_id)

Persona(persona_id=UUID('3df23b31-0e9d-515b-9aef-ed704641bbe2'), properties={'age': 30.0, 'language': 'English', 'favorite_genres': ['Sci-Fi', 'Horror', 'Mystery'], 'favorite_years': [1999.0, 1996.0, 2008.0, 2019.0]})

In [6]:
from weaviate.agents.classes import PersonaInteraction
from weaviate.collections.classes.filters import Filter

movie_titles = [
    "Avatar",
    "How It Ends",
    "Doctor Sleep",
    "The Howling",
    "A Nightmare on Elm Street",
    "The Howling",
    "High Life",
    "Magic Mike",
]

movies_collection = client.collections.get(PA_DEMO_COLLECTION)

movie_dict = {
    movie.properties["title"]: movie
    for movie in movies_collection.query.fetch_objects(
        filters=Filter.by_property("title").contains_any(movie_titles), limit=20
    ).objects
}

interactions = [
    PersonaInteraction(
        persona_id=persona_id, item_id=movie_dict["Avatar"].uuid, weight=0.5
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=movie_dict["Magic Mike"].uuid, weight=-0.9
    ),  # negative
]

In [7]:
pa.add_interactions(interactions=interactions)

In [8]:
interactions = [
    PersonaInteraction(
        persona_id=persona_id, item_id=movie_dict["How It Ends"].uuid, weight=0.7
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=movie_dict["Doctor Sleep"].uuid, weight=0.6
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=movie_dict["The Howling"].uuid, weight=0.5
    ),
    PersonaInteraction(
        persona_id=persona_id,
        item_id=movie_dict["A Nightmare on Elm Street"].uuid,
        weight=0.9,
    ),
    PersonaInteraction(
        persona_id=persona_id, item_id=movie_dict["High Life"].uuid, weight=0.75
    ),
]

In [9]:
pa.add_interactions(interactions=interactions)

## Personalized Queries

### No Agent Reranker

Fast, most basic personalized queries

- Uses vectors of interaction history only


In [10]:
response = pa.get_objects(persona_id, limit=50, use_agent_ranking=False)

In [11]:
from helpers import print_movie_response_details

print_movie_response_details(response, 5)

None
*****0*****
The Shining
Jack Torrance accepts a caretaker job at the Overlook Hotel, where he, along with his wife Wendy and their son Danny, must live isolated from the rest of the world for the winter. But they aren't prepared for the madness that lurks within.
['Horror', 'Thriller']
1980-05-23
vote_average: 8.2
vote_count: 14120
popularity: 57.144
original rank: 0, personalized rank: None
*****1*****
A-X-L
The life of a teenage boy is forever altered by a chance encounter with cutting edge military technology.
['Science Fiction', 'Action', 'Adventure', 'Family']
2018-08-23
vote_average: 6.3
vote_count: 988
popularity: 33.44
original rank: 1, personalized rank: None
*****2*****
Slumber
A sleep doctor tries to protect a family from a demon that feeds on people in their nightmares.
['Horror']
2017-12-01
vote_average: 5.3
vote_count: 286
popularity: 17.364
original rank: 2, personalized rank: None
*****3*****
The Shadow Effect
A young man's life is turned upside down when his viole

### With Reranker + Instruction + Filter

- The most complex personalized queries
- Uses vectors of interaction history and AI-based reranker
- Instructions used to guide the reranker
- Filters out items that are not relevant to the user


In [14]:
# With Reranker + Instruction + Filter
response = pa.get_objects(
    persona_id,
    limit=50,
    use_agent_ranking=True,
    instruction="Users favorite sub genre of horror is slasher and least favorite is space horror",
    filters=Filter.by_property("genres").contains_any(["Mystery", "Horror"]),
)

In [15]:
print_movie_response_details(response, 10)

Since you enjoy Horror, particularly slasher films, and have a preference for recent releases, we've highlighted movies that align with these interests. We've emphasized films with strong horror elements and avoided those with space horror themes, reflecting your preferences.
*****0*****
The Shining
Jack Torrance accepts a caretaker job at the Overlook Hotel, where he, along with his wife Wendy and their son Danny, must live isolated from the rest of the world for the winter. But they aren't prepared for the madness that lurks within.
['Horror', 'Thriller']
1980-05-23
vote_average: 8.2
vote_count: 14120
popularity: 57.144
original rank: 0, personalized rank: 0
*****1*****
Insidious
A family discovers that dark spirits have invaded their home after their son inexplicably falls into an endless sleep. When they reach out to a professional for help, they learn things are a lot more personal than they thought.
['Horror', 'Thriller']
2010-09-13
vote_average: 6.9
vote_count: 5385
popularity: 

## Compare results 

What happens if we look at results across different query methods?

In [18]:
from helpers import compare_genre_match_scores

query = pa.query(persona_id=persona_id, strength=0.95)  # fully personalized reranking
query2 = pa.query(persona_id=persona_id, strength=0.0)  # no reranking

response1 = query.hybrid("slasher", limit=50)
response2 = query2.hybrid("slasher", limit=50)

compare_genre_match_scores(
    [response1, response2],
    preferred_genres={"Science Fiction", "Horror", "Mystery"},
    top_n=10,
    response_labels=["Personalized", "No Personalization"],
)

Rank Personalized Title                      Personalized Genres                     Score   | No Personalization Title                No Personalization Genres               Score  
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
1    The Final                               Thriller, Horror                        1       | Chucky's Vacation Slides                Comedy, Horror                          1      
2    Scream                                  Horror, Mystery, Thriller               2       | Prom Night                              Horror, Thriller                        1      
3    Urban Legend                            Horror, Thriller                        1       | Monster                                 Crime, Drama                            0      
4    You're Next                             Horror, Thriller, Mystery               2  

We see that the personalized results are much more relevant to the user than the non-personalized results.